In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units as u
import pandas as pd
from scipy.interpolate import interp1d

In [2]:
from lsst.geom import Point2D, Point2I
from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
import lsst.afw.cameraGeom.utils as cgUtils
from lsst.daf.persistence import Butler, NoResults

In [3]:
import lsst.syseng.throughputs as st
from lsst.sims.photUtils import PhotometricParameters, Bandpass, LSSTdefaults
from lsst.sims.utils import angularSeparation

In [32]:
#rname = 'R10'
#rname = 'R22'
#rname = 'R01'
#rname = 'R11'
#rname = 'R20'
#rname = 'R21'
rname = 'R30'
#rname = 'R12'
#rname = 'R02'
#rname = 'R31'
#rname = 'R03'
#rname = 'R34'
#rname = 'R13'
#rname = 'R23'
#rname = 'R14'
#rname = 'R32'
#rname = 'R24'
#rname = 'R41'
#rname = 'R42'
#rname = 'R33'
#rname = 'R43'

In [33]:
# mapping based on 
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=LSSTCAM&title=Raft+Delivery+and+Acceptance+Testing+Status
dd = pd.read_csv('raftInstall.csv',index_col=0)
useDefault = False
try:
    if np.isnan(dd.rtm[rname]):
        useDefault = True
        print('no data yet, use default')
except TypeError:
    pass

### Default photometric parameters, as used in standard m5 calculations
Note that effarea is not in this list here, because it varies with field.

The read noise is not in this list either, because it varies by amp.

In [34]:
exptime=15 
nexp=2
othernoise=0 
darkcurrent=0.2
X=1.0

In [35]:
#we do not need this cell to calculate m5, but these FWHMeff are the default values we use actually
lsstDefaults = LSSTdefaults()

### Set up throughputs for hardware and atmosphere. Use the default detector QE as in syseng_throughput for now

In [36]:
# Add losses to each component?
addLosses = True
defaultDirs = st.setDefaultDirs()
defaultDirs

{'detector': '/home/bxin/notebooks/syseng_throughputs/components/camera/detector/joint_minimum',
 'lens1': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens1',
 'lens2': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens2',
 'lens3': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens3',
 'filters': '/home/bxin/notebooks/syseng_throughputs/components/camera/filters',
 'mirror1': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror1',
 'mirror2': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror2',
 'mirror3': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror3',
 'atmosphere': '/home/bxin/notebooks/syseng_throughputs/siteProperties'}

In [37]:
atmos = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
mirror1 = st.buildMirror(defaultDirs['mirror1'], addLosses)
mirror2 = st.buildMirror(defaultDirs['mirror2'], addLosses)
mirror3 = st.buildMirror(defaultDirs['mirror3'], addLosses)
lens1 = st.buildLens(defaultDirs['lens1'], addLosses)
lens2 = st.buildLens(defaultDirs['lens2'], addLosses)
lens3 = st.buildLens(defaultDirs['lens3'], addLosses)
filters = st.buildFilters(defaultDirs['filters'], addLosses)

vendor = dd.vendor[rname]
vendorDir = defaultDirs['detector']+'/../'+vendor.lower()
addLosses = False 
detector0 = st.buildDetector(vendorDir, addLosses) #design QE from this vendor
detector = Bandpass()

In [38]:
for f in filters:
    print(f, lsstDefaults.FWHMeff(f), lsstDefaults.m5(f))

r 0.83 24.43
i 0.8 24.0
u 0.92 23.68
z 0.78 24.45
g 0.87 24.89
y 0.76 22.6


### Butler access to the QE data

In [39]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/lsstcam/CALIB" 
print(DATADIR)
butler = Butler(DATADIR)
cam = butler.get('camera')

#This is no longer needed after tickets/DM-22605
#from lsst.obs.lsst.lsstCamMapper import LsstCamMapper
#mapper = LsstCamMapper()
#lsstcam = mapper.camera

/home/bxin/lsst_stack/obs_lsst/lsstcam/CALIB


### Prepare vignetting function

In [40]:
# below we use v3.11 values
vfile = f"{os.environ['HOME']}/notebooks/f_factors/data/vignettingF.txt"
M1D = 8.36 #clear aperture as in Optical design
aa = np.loadtxt(vfile, skiprows=12)
vr = aa[:,0]
vv = aa[:,1]

### Create the dataframes for this raft

In [41]:
#filterlist = tuple([s for s in filters]+['fS']) # this would be in different order than below
filterlist = ['u', 'g', 'r', 'i', 'z', 'y', 'fS']
alist = ('raDeg', 'decDeg', 'radDeg', 'effarea', 'readnoise', 'gain', 'saturation')
detectors = []
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    detectors.append(det.getName())
adf = pd.DataFrame(index=alist, columns=detectors, dtype=object)
m5df = pd.DataFrame(index=filterlist, columns=detectors, dtype=object)
Tdf = pd.DataFrame(index=filterlist[:6], columns=detectors, dtype=object)
Sdf = pd.DataFrame(index=filterlist[:6], columns=detectors, dtype=object)

In [42]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    key = rname+'_'+dname
    raDeg = {}
    decDeg = {}
    readnoise = {}
    gain = {}
    saturation = {}
    for amp in det:
        i = amp.getName()
        amp_point = amp.getBBox().getCenter()
        raDec = det.transform(amp_point, PIXELS, FIELD_ANGLE) 
        [raDeg[i], decDeg[i]] = np.degrees(raDec)
        gain[i] = amp.getGain()
        saturation[i] = amp.getSaturation()
        #print(key, i, raDec, amp.getGain(), amp.getSaturation())
        if useDefault:
            readnoise[i] = 8.8
        else:
            readnoise[i] = amp.getReadNoise()
    adf[key].loc['raDeg'] = list(raDeg.values())
    adf[key].loc['decDeg'] = list(decDeg.values())
    adf[key].loc['readnoise'] = list(readnoise.values())
    adf[key].loc['gain'] = list(gain.values())
    adf[key].loc['saturation'] = list(saturation.values())
    
    #effective area
    radius = angularSeparation(0., 0., adf[key]['raDeg'], adf[key]['decDeg'])
    adf[key].loc['radDeg'] = list(radius)
    adf[key].loc['effarea'] = list(np.interp(radius, vr, vv)*np.pi*(M1D/2)**2)

In [48]:
adf['R30_S00'].loc['raDeg']

[-1.7454999999999998,
 -1.7170555555555553,
 -1.6886111111111108,
 -1.6601666666666663,
 -1.631722222222222,
 -1.603277777777778,
 -1.5748333333333335,
 -1.546388888888889,
 -1.5463888888888888,
 -1.5748333333333333,
 -1.603277777777778,
 -1.631722222222222,
 -1.6601666666666666,
 -1.6886111111111108,
 -1.7170555555555556,
 -1.7454999999999998]

In [49]:
adf

,R30_S00,R30_S01,R30_S02,R30_S10,R30_S11,R30_S12,R30_S20,R30_S21,R30_S22
raDeg,"[-1.7454999999999998, -1.7170555555555553, -1....","[-1.5106111111111113, -1.4821666666666669, -1....","[-1.2757777777777775, -1.2473333333333332, -1....","[-1.7452222222222222, -1.7167777777777777, -1....","[-1.510111111111111, -1.4816666666666665, -1.4...","[-1.275222222222222, -1.2467777777777778, -1.2...","[-1.7444999999999997, -1.7160555555555552, -1....","[-1.5095555555555558, -1.481111111111111, -1.4...","[-1.2747777777777778, -1.246333333333333, -1.2..."
decDeg,"[0.5301111111111111, 0.530111111111111, 0.5301...","[0.5293888888888889, 0.5293888888888889, 0.529...","[0.5291111111111111, 0.5291111111111111, 0.529...","[0.7649444444444443, 0.7649444444444442, 0.764...","[0.7644999999999998, 0.7645, 0.764499999999999...","[0.7639444444444444, 0.7639444444444444, 0.763...","[0.9998333333333332, 0.9998333333333334, 0.999...","[0.9993888888888888, 0.9993888888888888, 0.999...","[0.9989444444444445, 0.9989444444444445, 0.998..."
radDeg,"[1.8241987547351064, 1.7970012349346327, 1.769...","[1.6006666064089317, 1.5738513609370899, 1.547...","[1.3811304198896208, 1.354900285336899, 1.3287...","[1.9054552289721105, 1.8794389627137005, 1.853...","[1.6925603316368223, 1.6672330581646349, 1.642...","[1.4865081428719802, 1.4621811862227192, 1.438...","[2.010631246653017, 1.9860046381411725, 1.9614...","[1.810332845294283, 1.7866856990026423, 1.7631...","[1.6195010707515631, 1.5972098735025786, 1.575..."
effarea,"[26.260597360765562, 28.360343811926974, 30.03...","[31.929010100533198, 32.087499843492644, 32.23...","[32.9731312573011, 33.09425270244143, 33.16341...","[20.74914185908972, 22.21510740371344, 23.8990...","[31.412432065239265, 31.55391266611826, 31.685...","[32.54369853507912, 32.658885840115644, 32.785...","[9.940373888470013, 13.156576281203105, 15.645...","[27.364091117008773, 29.064084966317846, 30.37...","[31.78401324653579, 31.950736119364468, 32.080..."
readnoise,"[190.839, 4.88042, 4.9657, 4.85721, 4.85988, 4...","[4.77568, 4.72945, 4.77169, 4.81156, 4.84422, ...","[4.8144, 4.82478, 4.85863, 4.85427, 4.99898, 4...","[4.78948, 4.68746, 4.77975, 5.08454, 4.86799, ...","[4.82368, 4.86618, 4.81932, 4.81477, 4.93418, ...","[5.03364, 5.14437, 5.17038, 5.2416, 5.24627, 5...","[4.80675, 4.78531, 4.77967, 4.83558, 4.814, 4....","[4.84613, 4.99574, 4.88679, 4.79734, 4.83482, ...","[5.14104, 5.00893, 4.92058, 5.13594, 5.22033, ..."
gain,"[55.8557, 0.773209, 0.767844, 0.762802, 0.7628...","[0.747689, 0.745674, 0.743374, 0.749392, 0.753...","[0.786233, 0.784979, 0.788977, 0.791223, 0.792...","[0.719295, 0.724405, 0.726687, 0.725835, 0.720...","[0.756738, 0.762915, 0.766955, 0.763714, 0.771...","[0.805771, 0.806143, 0.809563, 0.806923, 0.810...","[0.765164, 0.772919, 0.776382, 0.769116, 0.777...","[0.745813, 0.747989, 0.75061, 0.745198, 0.7487...","[0.751423, 0.757567, 0.761949, 0.768608, 0.767..."
saturation,"[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285..."


In [50]:
#adf['R32_S11'].loc['readnoise']

In [51]:
ampList = list(raDeg.keys())

In [52]:
# this is needed if there are only 6 QE measurements per amp
idx = np.where(detector0.sb>0.01)
idx1=idx[0][0]-1
idx2=idx[0][-1]+1

x1 = detector0.wavelen[idx1]
y1 = detector0.sb[idx1]
x2 = detector0.wavelen[idx2]
y2 = detector0.sb[idx2]

In [53]:
m5SRD = np.array([23.9, 25.0, 24.7, 24.0, 23.3, 22.1])
#m5SRDmin = []
# Nv1 from SRD table 24
Nv1 = np.array([56, 80, 184, 184, 160, 160])
omega = Nv1/sum(Nv1)
fidx = 'ugrizy' #very important!

In [54]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
        
    vendor = det.getSerial()[:3].lower()
    assert useDefault or dd.vendor[rname].lower() == vendor
    vendorDir = defaultDirs['detector']+'/../'+vendor
    print('Calculating m5 for %s_%s'%(rname,dname))
    
    key = rname+'_'+dname
    for f in filterlist:
        m5df[key][f] = [-1.]*len(ampList)
    for f in filterlist[:6]:
        Tdf[key][f] = [-1.]*len(ampList)
        Sdf[key][f] = [-1.]*len(ampList)
        
    for amp in det:
        
        try:
            qe_curve = butler.get('qe_curve', raftName=rname, detectorName=dname, calibDate='1970-01-01T00:00:00')
            wavelen = detector0.wavelen 

            k = amp.getName()
            if len(qe_curve.data[k][0])>10:
                wlen = qe_curve.data[k][0]
                eff = qe_curve.data[k][1]
                f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='quadratic')
            else:
                aa = np.append(x1, qe_curve.data[k][0].value)
                aa = np.append(aa, x2)
                wlen = aa * qe_curve.data[k][0].unit

                aa = np.append(y1, qe_curve.data[k][1].value)
                aa = np.append(aa, y2)
                eff = aa * qe_curve.data[k][1].unit
                f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='slinear')#quadratic causes overshoot
            
            sb = f(wavelen)*0.01
            #alternatively we could do (only for >10 QE measurements)
            #amp_point = amp.getBBox().getCenter()
            #sb = qe_curve.evaluate(det, amp_point, wavelen* u.nm, kind='quadratic').value*.01 #unit was percent in CALIB data

            sb[np.isnan(sb)] = 0
            if np.max(sb)>1.5:
                print('These seem too LARGE ', k)
                print(np.max(sb))
                sb = 0
            if np.max(sb)<0.2: #3 dead channels, 1 out of each of R01, R10, and R30; see camera confluence page table
                print('dead channel: %s %s, max sb = %.2f'%(key, amp.getName(), np.max(sb)))
                continue;
                
            detector.setBandpass(wavelen, sb)
                
            #detector losses  
            #os.listdir(vendorDir)
            detLosses = Bandpass()
            detLosses.readThroughput(os.path.join(vendorDir, '%s_Losses/det_Losses.dat' % (vendor)))
                
            #build hardware and system
            hardware = {}
            system = {}
            for f in filters:
                sb = mirror1.sb * mirror2.sb *mirror3.sb
                sb *= lens1.sb * lens2.sb * lens3.sb * filters[f].sb
                sb *= detector.sb * detLosses.sb
                
                hardware[f] = Bandpass()
                hardware[f].setBandpass(wavelen, sb)
                system[f] = Bandpass()
                system[f].setBandpass(wavelen, sb * atmos.sb)
                
        except NoResults:
            if not useDefault:
                print('No results found for this detector')
            assert useDefault
            
            hardware, system = st.buildHardwareAndSystem(defaultDirs)
            
        #calculate m5      
        iamp = ampList.index(amp.getName())
        effarea = adf[key]['effarea'][iamp]*100**2 #convert to cm^2
        readnoise = adf[key]['readnoise'][iamp]
        
        m5 = st.makeM5(hardware, system, darksky=None, 
                    exptime=15, nexp=2, readnoise=readnoise, othernoise=0, darkcurrent=0.2,
                    effarea=effarea, X=1.0)
        for f in filters:
            m5df[key][f][iamp] = m5.m5[f]
            Tdf[key][f][iamp] = m5.Tb[f]
            Sdf[key][f][iamp] = m5.Sb[f]
        m5amp = np.array([m5.m5[f] for f in fidx])
        if np.all(m5amp>0):
            m5df[key]['fS'][iamp] = sum(omega*10**(0.8*(m5amp - m5SRD)))
        

Calculating m5 for R30_S00
dead channel: R30_S00 C10, max sb = 0.00
Calculating m5 for R30_S01
Calculating m5 for R30_S02
Calculating m5 for R30_S10
Calculating m5 for R30_S11
Calculating m5 for R30_S12
Calculating m5 for R30_S20
Calculating m5 for R30_S21
Calculating m5 for R30_S22


In [55]:
Sdf

,R30_S00,R30_S01,R30_S02,R30_S10,R30_S11,R30_S12,R30_S20,R30_S21,R30_S22
u,"[-1.0, 0.06154937586510727, 0.0626040150907606...","[0.06037149337873526, 0.06042585549765393, 0.0...","[0.0660715469507539, 0.0663100243845133, 0.066...","[0.06400974506365562, 0.064225939030313, 0.064...","[0.06271665714470147, 0.06315560314888392, 0.0...","[0.06269230710481183, 0.06302283678340426, 0.0...","[0.05635422107504149, 0.056854755712280165, 0....","[0.05337752577786105, 0.054419156054395276, 0....","[0.06564604106058808, 0.06593293795859134, 0.0..."
g,"[-1.0, 0.1615226580624046, 0.16225743283085592...","[0.16041133112256878, 0.16023955809914517, 0.1...","[0.16408266705841365, 0.1640299521659438, 0.16...","[0.1622343388239926, 0.16222468641696788, 0.16...","[0.1617764348312566, 0.16205074263712316, 0.16...","[0.16375941892021836, 0.16382646651369914, 0.1...","[0.15965076487790222, 0.15969998110628833, 0.1...","[0.15917875064617065, 0.1599292480875684, 0.16...","[0.16390926270406406, 0.16423737354117202, 0.1..."
r,"[-1.0, 0.1259917844450727, 0.126290587591704, ...","[0.1260650727411054, 0.12588431194893387, 0.12...","[0.1267646070024614, 0.1266250272252422, 0.126...","[0.12631499858325845, 0.12624110856053325, 0.1...","[0.12679890238343283, 0.1268962516178687, 0.12...","[0.12755675536205996, 0.12749719391404754, 0.1...","[0.1263607164093468, 0.12621272866177524, 0.12...","[0.12661332889518617, 0.12687613703406841, 0.1...","[0.1272278337237196, 0.12742947514027744, 0.12..."
i,"[-1.0, 0.08909186229518212, 0.0893175961506185...","[0.0890685104313849, 0.08890999510536848, 0.08...","[0.08944150459011214, 0.0894223410453633, 0.08...","[0.08895197005201079, 0.08902473759883549, 0.0...","[0.08918112065692196, 0.08922018780154144, 0.0...","[0.08941588028418557, 0.08946033113910305, 0.0...","[0.08913851923094988, 0.0891818545830674, 0.08...","[0.08996436389296032, 0.09012578727054736, 0.0...","[0.08954264544761932, 0.08977135648521711, 0.0..."
z,"[-1.0, 0.058413158139835546, 0.058543327156238...","[0.0582850877360844, 0.058173536407611054, 0.0...","[0.05851665697381591, 0.05855085928741756, 0.0...","[0.05822237280652386, 0.058305719591582865, 0....","[0.05809172095929657, 0.05811628747865292, 0.0...","[0.05813440386085391, 0.05821643152311349, 0.0...","[0.058318488215753035, 0.058339839125876924, 0...","[0.05895898261596234, 0.05905151191059104, 0.0...","[0.05837246833807129, 0.058611004534876866, 0...."
y,"[-1.0, 0.028387703008441026, 0.028439531653337...","[0.028247899604630304, 0.028260852722787975, 0...","[0.028516912047298493, 0.028529138436077846, 0...","[0.02822577306144989, 0.02821553482837537, 0.0...","[0.028083564176793908, 0.02811853468228403, 0....","[0.028071344922707624, 0.02809230425950571, 0....","[0.028286939354991106, 0.028247018696968974, 0...","[0.028394058260150422, 0.02844639954745732, 0....","[0.028307004538634765, 0.028458357332936442, 0..."


In [56]:
m5df

,R30_S00,R30_S01,R30_S02,R30_S10,R30_S11,R30_S12,R30_S20,R30_S21,R30_S22
u,"[-1.0, 24.123668891641955, 24.17461418422136, ...","[24.20737604715376, 24.215210773222612, 24.219...","[24.296886104682454, 24.301497921541717, 24.30...","[23.912388494707827, 23.97915744962063, 24.034...","[24.218868094232235, 24.224374877487474, 24.23...","[24.2311618956083, 24.22966068808654, 24.23395...","[23.16119265242097, 23.425318191130472, 23.585...","[23.989017644959706, 24.039553393272175, 24.09...","[24.235432336529517, 24.252356615150383, 24.26..."
g,"[-1.0, 24.876505322556444, 24.912914402529893,...","[24.947224230612164, 24.95058597843116, 24.954...","[24.980431470245613, 24.982282507319542, 24.98...","[24.68562680887138, 24.731914854838287, 24.776...","[24.941511069119418, 24.94441175801808, 24.948...","[24.96617890485915, 24.966229712315084, 24.968...","[24.180167398784562, 24.37558879427184, 24.492...","[24.8453675915299, 24.882131020561204, 24.9124...","[24.94988329212161, 24.957267883548134, 24.960..."
r,"[-1.0, 24.393942816942136, 24.427881213641534,...","[24.464478055435137, 24.46702322847311, 24.470...","[24.485987717526573, 24.487364120084024, 24.48...","[24.212301878621865, 24.254100414890203, 24.29...","[24.457851842049486, 24.46043339964731, 24.463...","[24.479607777955678, 24.480146488864236, 24.48...","[23.7588098904368, 23.935200517458256, 24.0415...","[24.376250183429022, 24.410863085493386, 24.43...","[24.463191744683026, 24.468686369966264, 24.47..."
i,"[-1.0, 23.926338918649556, 23.960025662027725,...","[23.995276130465967, 23.997492839668123, 24.00...","[24.015616870579954, 24.017408463916823, 24.01...","[23.745800168528675, 23.787083861771052, 23.82...","[23.9867085941774, 23.989206304597687, 23.9920...","[24.00669246370834, 24.007962192747307, 24.009...","[23.304977727409465, 23.476702354213636, 23.58...","[23.911386299623253, 23.945717484526657, 23.97...","[23.992285506659652, 23.997793649144135, 24.00..."
z,"[-1.0, 23.32596613427271, 23.359294884015107, ...","[23.392381209292694, 23.394302982866787, 23.39...","[23.412642719522033, 23.41500881946762, 23.416...","[23.146674124376148, 23.187450283670945, 23.22...","[23.38103078652148, 23.383516380915097, 23.386...","[23.400446235681194, 23.402595109808203, 23.40...","[22.717865687561215, 22.88467021342027, 22.985...","[23.31160438935283, 23.345655396335708, 23.370...","[23.38918886225529, 23.395171284377625, 23.398..."
y,"[-1.0, 22.370494110640323, 22.40341391681988, ...","[22.43475026673129, 22.437909790706023, 22.442...","[22.457867410373552, 22.46013077529474, 22.461...","[22.190420943547338, 22.22976432646731, 22.270...","[22.422165393231953, 22.425166054322567, 22.42...","[22.440786967445316, 22.44267507599814, 22.445...","[21.765857606191148, 21.92957703297384, 22.029...","[22.350607153724162, 22.384747775480033, 22.40...","[22.431436184245086, 22.438051473887846, 22.44..."
fS,"[-1.0, 1.025257025377821, 1.0942731781396169, ...","[1.1638190784431957, 1.1706085604210654, 1.179...","[1.2295403589589986, 1.2348187138331195, 1.237...","[0.7304441114998044, 0.7913713867751224, 0.856...","[1.1472297909812794, 1.153612571463322, 1.1614...","[1.1885735296074564, 1.191107826493704, 1.1968...","[0.311752601025751, 0.4310364303188665, 0.5249...","[0.9722206055068215, 1.038467587861686, 1.0934...","[1.1655037953582963, 1.1813416327169106, 1.190..."


In [57]:
dfDir = os.path.join('m5_output', rname)

In [58]:
if not os.path.exists(dfDir):
    os.mkdir(dfDir)
dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
adf.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
m5df.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'Tdf_%s.csv'%rname)
Tdf.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'Sdf_%s.csv'%rname)
Sdf.to_csv(dfPath)

### Make sure we can read out the dataframes correctly

In [59]:
from ast import literal_eval

In [60]:
dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
df = pd.read_csv(dfPath, index_col=0)

In [61]:
df['%s_S00'%rname].apply(literal_eval)#['readnoise'][0]

raDeg         [-1.7454999999999998, -1.7170555555555553, -1....
decDeg        [0.5301111111111111, 0.530111111111111, 0.5301...
radDeg        [1.8241987547351064, 1.7970012349346327, 1.769...
effarea       [26.260597360765562, 28.360343811926974, 30.03...
readnoise     [190.839, 4.88042, 4.9657, 4.85721, 4.85988, 4...
gain          [55.8557, 0.773209, 0.767844, 0.762802, 0.7628...
saturation    [142857.0, 142857.0, 142857.0, 142857.0, 14285...
Name: R30_S00, dtype: object

In [62]:
adf

,R30_S00,R30_S01,R30_S02,R30_S10,R30_S11,R30_S12,R30_S20,R30_S21,R30_S22
raDeg,"[-1.7454999999999998, -1.7170555555555553, -1....","[-1.5106111111111113, -1.4821666666666669, -1....","[-1.2757777777777775, -1.2473333333333332, -1....","[-1.7452222222222222, -1.7167777777777777, -1....","[-1.510111111111111, -1.4816666666666665, -1.4...","[-1.275222222222222, -1.2467777777777778, -1.2...","[-1.7444999999999997, -1.7160555555555552, -1....","[-1.5095555555555558, -1.481111111111111, -1.4...","[-1.2747777777777778, -1.246333333333333, -1.2..."
decDeg,"[0.5301111111111111, 0.530111111111111, 0.5301...","[0.5293888888888889, 0.5293888888888889, 0.529...","[0.5291111111111111, 0.5291111111111111, 0.529...","[0.7649444444444443, 0.7649444444444442, 0.764...","[0.7644999999999998, 0.7645, 0.764499999999999...","[0.7639444444444444, 0.7639444444444444, 0.763...","[0.9998333333333332, 0.9998333333333334, 0.999...","[0.9993888888888888, 0.9993888888888888, 0.999...","[0.9989444444444445, 0.9989444444444445, 0.998..."
radDeg,"[1.8241987547351064, 1.7970012349346327, 1.769...","[1.6006666064089317, 1.5738513609370899, 1.547...","[1.3811304198896208, 1.354900285336899, 1.3287...","[1.9054552289721105, 1.8794389627137005, 1.853...","[1.6925603316368223, 1.6672330581646349, 1.642...","[1.4865081428719802, 1.4621811862227192, 1.438...","[2.010631246653017, 1.9860046381411725, 1.9614...","[1.810332845294283, 1.7866856990026423, 1.7631...","[1.6195010707515631, 1.5972098735025786, 1.575..."
effarea,"[26.260597360765562, 28.360343811926974, 30.03...","[31.929010100533198, 32.087499843492644, 32.23...","[32.9731312573011, 33.09425270244143, 33.16341...","[20.74914185908972, 22.21510740371344, 23.8990...","[31.412432065239265, 31.55391266611826, 31.685...","[32.54369853507912, 32.658885840115644, 32.785...","[9.940373888470013, 13.156576281203105, 15.645...","[27.364091117008773, 29.064084966317846, 30.37...","[31.78401324653579, 31.950736119364468, 32.080..."
readnoise,"[190.839, 4.88042, 4.9657, 4.85721, 4.85988, 4...","[4.77568, 4.72945, 4.77169, 4.81156, 4.84422, ...","[4.8144, 4.82478, 4.85863, 4.85427, 4.99898, 4...","[4.78948, 4.68746, 4.77975, 5.08454, 4.86799, ...","[4.82368, 4.86618, 4.81932, 4.81477, 4.93418, ...","[5.03364, 5.14437, 5.17038, 5.2416, 5.24627, 5...","[4.80675, 4.78531, 4.77967, 4.83558, 4.814, 4....","[4.84613, 4.99574, 4.88679, 4.79734, 4.83482, ...","[5.14104, 5.00893, 4.92058, 5.13594, 5.22033, ..."
gain,"[55.8557, 0.773209, 0.767844, 0.762802, 0.7628...","[0.747689, 0.745674, 0.743374, 0.749392, 0.753...","[0.786233, 0.784979, 0.788977, 0.791223, 0.792...","[0.719295, 0.724405, 0.726687, 0.725835, 0.720...","[0.756738, 0.762915, 0.766955, 0.763714, 0.771...","[0.805771, 0.806143, 0.809563, 0.806923, 0.810...","[0.765164, 0.772919, 0.776382, 0.769116, 0.777...","[0.745813, 0.747989, 0.75061, 0.745198, 0.7487...","[0.751423, 0.757567, 0.761949, 0.768608, 0.767..."
saturation,"[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285...","[142857.0, 142857.0, 142857.0, 142857.0, 14285..."


In [63]:
adf['R30_S00'].loc['raDeg']

[-1.7454999999999998,
 -1.7170555555555553,
 -1.6886111111111108,
 -1.6601666666666663,
 -1.631722222222222,
 -1.603277777777778,
 -1.5748333333333335,
 -1.546388888888889,
 -1.5463888888888888,
 -1.5748333333333333,
 -1.603277777777778,
 -1.631722222222222,
 -1.6601666666666666,
 -1.6886111111111108,
 -1.7170555555555556,
 -1.7454999999999998]

In [29]:
dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
df = pd.read_csv(dfPath, index_col=0)

In [30]:
df['%s_S00'%rname].apply(literal_eval)#['readnoise'][0]

u     [24.169428234328322, 24.175232683141367, 24.18...
g     [24.962417962642938, 24.962847405772102, 24.96...
r     [24.491807175628303, 24.491433982527028, 24.49...
i     [24.018125070373934, 24.018613896545656, 24.01...
z     [23.405258382789434, 23.40641239792137, 23.405...
y     [22.506148770771198, 22.506691752618433, 22.50...
fS    [1.2313155323812248, 1.233602903209286, 1.2350...
Name: R32_S00, dtype: object

In [31]:
#df['R01_S11']['u']